In [15]:
import keras
import numpy as np
import os
import sys
sys.path.append("../../../ecg")
import scipy.stats as sst

import util
import load

In [28]:
model_path = "/deep/group/awni/ecg_models/default/1527815883-932/0.350-0.885-022-0.324-0.886.hdf5"
data_path = "../test.json"

data = load.load_dataset(data_path)
preproc = util.load(os.path.dirname(model_path))
model = keras.models.load_model(model_path)

100%|██████████| 852/852 [00:00<00:00, 931.78it/s] 


In [29]:
probs = []
labels = []
for x, y  in zip(*data):
    x, y = preproc.process([x], [y])
    probs.append(model.predict(x))
    labels.append(y)

In [30]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [31]:
import sklearn.metrics as skm
report = skm.classification_report(
            ground_truth, preds,
            target_names=preproc.classes,
            digits=3)
print(report)

             precision    recall  f1-score   support

          A      0.843     0.797     0.819        74
          N      0.906     0.941     0.923       512
          O      0.815     0.791     0.803       239
          ~      0.556     0.370     0.444        27

avg / total      0.864     0.869     0.865       852

